In [1]:
import torch
from torch.utils.data.dataset import Dataset
from datasets import load_dataset 
from sklearn.model_selection import train_test_split 

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from sklearn.model_selection import train_test_split

# Example data. 
# In reality, the strings are usually longer and there are 11 possible classes
model_name = "prajjwal1/bert-tiny"
dataset = load_dataset("jigsaw_toxicity_pred", data_dir="/workspaces/LLM-Experimentation-Capstone/00_source_data/jigsaw_toxicity")
dataset = dataset['train'][0:250]
# Example data. 
# In reality, the strings are usually longer and there are 11 possible classes
texts = dataset['comment_text']

labels = [[float(x), float(y), float(z), float(a), float(b), float(c)] for x, y, z, a, b, c in zip(dataset['toxic'], dataset['severe_toxic'], dataset['obscene'], 
                                                   dataset['threat'], dataset['insult'], dataset['identity_hate'])]


train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    texts, labels, test_size=0.33, random_state=42
)

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=512)
eval_encodings = tokenizer(eval_texts, padding="max_length", truncation=True, max_length=512)

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TextClassifierDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextClassifierDataset(train_encodings, train_labels)
eval_dataset = TextClassifierDataset(eval_encodings, eval_labels)

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    problem_type="multi_label_classification",
    num_labels=6
)

training_arguments = TrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1024,
    per_device_eval_batch_size=32,
    num_train_epochs=50,
)

trainer = Trainer(
    model=model.to('cpu'),
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.623465
2,No log,0.617433
3,No log,0.612322
4,No log,0.607782
5,No log,0.603548
6,No log,0.599459
7,No log,0.595560
8,No log,0.591868
9,No log,0.588255
10,No log,0.584718


TrainOutput(global_step=50, training_loss=0.5453610610961914, metrics={'train_runtime': 9.5592, 'train_samples_per_second': 873.505, 'train_steps_per_second': 5.231, 'total_flos': 10621802803200.0, 'train_loss': 0.5453610610961914, 'epoch': 50.0})

In [4]:
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_tokenizer/")

('./final_tokenizer/tokenizer_config.json',
 './final_tokenizer/special_tokens_map.json',
 './final_tokenizer/vocab.txt',
 './final_tokenizer/added_tokens.json',
 './final_tokenizer/tokenizer.json')